In [2]:
import numpy as np
import os
import os.path
import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand
import nglview as nv
import imolecule
from ipywidgets import Label, Layout, Button, Box, HBox, VBox, Checkbox, FloatText, Text, Dropdown, Output
from IPython.display import display, clear_output
import readFiles
import createBonds
import createDreidingTypes
import runData4Lammps

%use ovito-2.8.0
%use lammps-31Mar17

<IPython.core.display.Javascript object>

In [3]:
def conect_in_pdb(infile):
    
    with open(infile) as pdb_file:    
        for line in pdb_file: 
            if line.startswith('CONECT'):        
                return True
            
        return False   

In [4]:
def run_all(ignore):
    
    download.w.disabled = True
    clear_output()
    display(buttom)
    infile = file.list()[0]
    #infile = './data/' + file.list()[0]
    bondscale = bondB.value
    lattice_in_file = checkBox.value
    ffield = ffieldB.value
    charge = chargeB.value
        
    boundaries = np.zeros(9)
    boundaries[0] = 0
    boundaries[1] = x.value
    boundaries[2] = 0
    boundaries[3] = y.value
    boundaries[4] = 0
    boundaries[5] = z.value
    boundaries[6] = a.value
    boundaries[7] = b.value
    boundaries[8] = g.value
    
    extension = os.path.splitext(infile)[1]
    infile_is_pdb = False
    
    if extension == '.pdb' and conect_in_pdb(infile): 
        print('Connectivity already in {}. Copying information ...'.format(infile))
        !cp {infile} ./bonds/connected_pdb.pdb
        outfilelmpdat, outfilepdb = readFiles.convert_structure(infile)
        boundaries = createBonds.simulation_box(outfilelmpdat, lattice_in_file, boundaries)
        infile_is_pdb = True
        
    else:
        print('Creating bonds ...')
        outfilelmpdat, outfilepdb = readFiles.convert_structure(infile)
        createBonds.create_bonds(outfilelmpdat, outfilepdb, bondscale, lattice_in_file, boundaries)

        
    outputName = infile.split('/')[-1].split('.')[0]    
    createDreidingTypes.create_dreiding_types('./bonds/connected_pdb.pdb', outputName, infile_is_pdb, ffield)
    print('Getting information...')
    new_typing_command, data4lammps_command = runData4Lammps.run_data4lammps(charge, infile_is_pdb, boundaries, ffield)
    #'Gasteiger'
    runCommand(new_typing_command)
    runCommand(data4lammps_command)
    runCommand('rm *index *name',stream=False)
    download.w.disabled = False

# LAMMPS Data File Generator

### This tool generates all necessary input files for LAMMPS simulations of molecular systems starting with an atomistic structure. 

#### Tool features:
 - Accepts a wide range of input formats (e.g. CIF, PDB, XYZ)
 - Works with periodic boundary conditions, even when covalent bonds cross boundariesThe tool uses OpenBabel to convert the initial files
 - Sets up energy expression for Dreiding and PCFF force fields (including automatic bond recognition and atom typing)
 - Assigns atomic changes using the Gasteiger method

<p style="color:red;font-size: 120%">Known bug: If resonance ring crosses the boundary, atoms will be incorrectly typed </p>

1) **Upload File** in any [format supported by OpenBabel](https://openbabel.org/wiki/Category:Formats).

In [5]:
def save_structure(name, outfilePDB):
    
    clear_output()
    display(file.input)
    
    outfilePDB = './bonds/pdbfile.pdb'
    structure_name = file.list()[0] 
    os.rename(structure_name, structure_name.replace(' ',''))
    structure_path = structure_name.replace(' ','')
    print('File uploaded succesfully')
    
    extension = os.path.splitext(structure_path)[1]
    if extension == '.cif':
        runCommand('obabel {0} -O {1} --fillUC strict --title convert ---errorlevel 1'.format(structure_path, outfilePDB), 
                   stream=True)
    else:
        runCommand('obabel {0} -O {1} --title convert ---errorlevel 1'.format(structure_path, outfilePDB), 
                   stream=True)
        
    imolecule.draw("./bonds/pdbfile.pdb",size=(1000, 300))#, camera_type="orthographic")    

file = FileUpload('File','Upload file', dir='./data', cb=save_structure, maxsize='1G')

In [6]:
display(file.input)

FileWidget()

2) **Additional inputs**

In [7]:
caption = Label(value='Note: Simulation cell parameters [A]')

checkBox = Checkbox(value = True, 
                    description = 'Simulation box included in file', 
                    disabled = False)

x = FloatText(value = 0, description = 'x:', disabled=False)
y = FloatText(value = 0, description = 'y:', disabled=False)
z = FloatText(value = 0, description = 'z:', disabled=False)

a = FloatText(value = 0, description =  r'\(\alpha:\)', disabled=False)
b = FloatText(value = 0, description =  r'\(\beta:\)', disabled=False)
g = FloatText(value = 0, description =  r'\(\gamma:\)', disabled=False)

user_boundaries = HBox([VBox([x, y, z]), VBox([a, b, g])])
boxs = VBox([checkBox, user_boundaries])
                  
def on_checkbox(change):
    if not change['new']:
        user_boundaries.layout.display = ''
    else:
        user_boundaries.layout.display = 'none'

checkBox.observe(on_checkbox, names='value')

ffieldB = Dropdown(options = ['Dreiding', 'PCFF'], 
                   value = 'Dreiding', 
                   description = 'Force field:',
                   disabled = False)

bondB = FloatText(value = 1.1, description = 'Bondscale:',
                  disabled = False)

chargeB = Dropdown(options = ['Gasteiger'],
                   value = 'Gasteiger', 
                   description = 'Charge:',
                   disabled = False)

box = VBox([ffieldB, bondB, chargeB])

Box = HBox([boxs,box], align_items = 'space-around')

display(Box, caption)

HBox(children=(VBox(children=(Checkbox(value=True, description='Simulation box included in file'), HBox(children=(VBox(children=(FloatText(value=0.0, description='x:'), FloatText(value=0.0, description='y:'), FloatText(value=0.0, description='z:'))), VBox(children=(FloatText(value=0.0, description='\\(\\alpha:\\)'), FloatText(value=0.0, description='\\(\\beta:\\)'), FloatText(value=0.0, description='\\(\\gamma:\\)'))))))), VBox(children=(Dropdown(description='Force field:', options=('Dreiding', 'PCFF'), value='Dreiding'), FloatText(value=1.1, description='Bondscale:'), Dropdown(description='Charge:', options=('Gasteiger',), value='Gasteiger')))))

Label(value='Note: Simulation cell parameters [A]')

In [8]:
runbut = Button(description='Create Data File')
runbut.on_click(run_all)

download = Download('LAMMPSDataFile.data',  
                          style = 'success',
                          tooltip = 'DOWNLOAD DATA FILE',
                          label = 'Download Datafile', 
                          icon = 'arrow-circle-down')
download.w.disabled = True

buttom = HBox([runbut, download.w])

display(buttom)

HBox(children=(Button(description='Create Data File', style=ButtonStyle()), Button(button_style='success', description='Download Datafile', disabled=True, icon='arrow-circle-down', style=ButtonStyle(), tooltip='DOWNLOAD DATA FILE')))